In [ ]:
import tools.globals as globals

a = []

if len(a) == 0:
    print("good")

In [ ]:
#選擇權下單範例

# 步驟
# 取得帳戶資訊
# 確立合約
# 確立order
# 送出order
 
# globals.position = globals.api.list_positions(globals.api.futopt_account)
# Offset.CLOSE: A closing offset (CO) order is a limit order that is executed at market close but can be placed anytime during the trading day. 

import shioaji as sj
from shioaji.constant import Action, OptionRight, StockPriceType, FuturesOrderType, FuturesOCType #選擇權下單，多匯入一個OptionRight常數

api = sj.Shioaji(simulation=True)
api.login(
    person_id='PAPIUSER06', 
    passwd='2222'
)

#contract = globals.api.Contracts.Options[option_code] #選擇權Contract
contract = globals.api.Contracts.Options.TXO.TXO202110017500C #選擇權Contract
order = globals.api.Order(
    action=Action.Buy,
    price=170, #價格
    quantity=2, #口數
    price_type=StockPriceType.LMT, #MKT: 市價 LMT: 限價
    order_type=FuturesOrderType.ROD,
    octype=FuturesOCType.Cover, #倉別，收盤時平倉
    OptionRight=OptionRight.Call, #選擇權類型
    account=globals.api.futopt_account #下單帳戶指定期貨帳戶
)

trade = globals.api.place_order(contract, order)
print(trade)